In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import BlackScholes as bs
import time
import variance_curve as vc
import ImpliedDrift as iD

from scipy.optimize import least_squares as ls
from rbergomi import rBergomi

In [2]:
def price(S, K, r, q, T):
    n = len(K)
    p = np.zeros(n)
    for i in range(n):
        p[i] = np.mean(np.maximum(S-K[i],0)*np.exp(-(r-q)*T))
    return p

In [3]:
dates = np.array(["23_01_23.csv", "24_01_23.csv", "25_01_23.csv", "26_01_23.csv", "27_01_23.csv",
                  "30_01_23.csv", "06_02_23.csv", "13_02_23.csv", "21_02_23.csv"])

spot = np.array(pd.read_csv("spot.csv").Spot).flatten()
moneyness = np.array([80.0,90.0,95.0,97.5,100.0,102.5,105.0,110.0,120.0])

index = 0
data = pd.read_csv(dates[index]); S0 = spot[index];
K = S0 * moneyness / 100

if index:
    maturities = np.array(data.Tenor).flatten()
    IV = np.array(data.drop(columns = ['Date', 'Tenor']))/100.

else:
    maturities = np.array(data['Exp Date']).flatten()
    IV = np.array(data.drop(columns = 'Exp Date'))/100.
    

maturities = maturities[maturities*365 <= 180]
nr = len(maturities); nc = len(K);
IV = IV[:nr]

In [4]:
nr = len(maturities); nc = len(moneyness);
N = 20000; n = 1000;
inp = np.array([0.1, 1.9, -0.9]) # Parameter array [H,eta,rho]
#inp = np.array([0.05659575,  2.13981363, -0.88309092])
bnds = ([.0001, 1, -0.999],[.5, 7, -.1])

rB_param = np.zeros(3)

start_time = time.time()

steps = np.ceil(n*maturities).astype(int)
T = maturities[-1]
np.random.seed(index)
r = iD.r(maturities); q = iD.q(maturities);

def h(x):

    t = time.time()
    
    np.random.seed(index)

    iv = np.zeros([nr,nc])
    H, eta, rho = x
    alpha = H - 0.5

    rB = rBergomi(n, N, T, alpha)
    xi = vc.variance_curve(rB.t[0], index)
    dW1 = rB.dW1(); dW2 = rB.dW2();    
    Ya = rB.Y(dW1)
    dZ = rB.dZ(dW1, dW2, rho)
    V = rB.V(Ya, xi, eta)

    S = rB.global_S(V, dZ, S0, steps, index)

    for i in range(nr):
        P = price(S[i,:], K, r[i], q[i], maturities[i])
        iv[i,:] = bs.BSImpliedVol(S0, K, maturities[i], r[i], q[i], P, Option_type = 1, toll = 1e-5)

    print(f'Function execution time: {time.time()-t: .0f} s')

    return iv

def f(x):
    return h(x).flatten() - IV.flatten()

result = ls(f, inp, bounds = bnds, max_nfev = 100, ftol = 1e-15, gtol = 1e-15, xtol = 1e-15)
rB_param = result.x
rB_vol = h(result.x)

total_time = (time.time() - start_time)/60

print(f'\nTotal execution time: {total_time: .0f} minutes')

rB_err = np.mean(abs(IV-rB_vol)/IV)*100

print(f"Relative mean error: {rB_err:.4f}%\n")

Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function execution time:  3 s
Function e

In [5]:
rB_param

array([ 0.05659575,  2.13981363, -0.88309092])

In [6]:
df = pd.DataFrame(rB_param.reshape(1,3), columns = ["H", "eta", "rho"])
df.to_csv("parameters_for_bayes.csv", index = False)

In [7]:
df

,H,eta,rho
0,0.056596,2.139814,-0.883091
